Hands-On Data Analysis with Pandas, Stefanie Molin

# Chapter 3 Exercises

11/20/23 

In [2]:
import pandas as pd
import numpy as np

## 1. With the earthquakes.csv file, select all the earthquakes in Japan with a magnitude of 4.9 or greater using the mb magnitude type.

In [3]:
quakes = pd.read_csv('exercises/earthquakes.csv')
quakes.head()

,mag,magType,time,place,tsunami,parsed_place
0,1.35,ml,1539475168010,"9km NE of Aguanga, CA",0,California
1,1.29,ml,1539475129610,"9km NE of Aguanga, CA",0,California
2,3.42,ml,1539475062610,"8km NE of Aguanga, CA",0,California
3,0.44,ml,1539474978070,"9km NE of Aguanga, CA",0,California
4,2.16,md,1539474716050,"10km NW of Avenal, CA",0,California


In [30]:
quakes.query(
    'parsed_place == "Japan"'
    'and magType == "mb"' 
    'and mag >= 4.9'
)[['mag', 'magType', 'place']]

,mag,magType,place
1563,4.9,mb,"293km ESE of Iwo Jima, Japan"
2576,5.4,mb,"37km E of Tomakomai, Japan"
3072,4.9,mb,"15km ENE of Hasaki, Japan"
3632,4.9,mb,"53km ESE of Hitachi, Japan"


---

## 2. Create bins for each full number of earthquake magnitude 
For instance, the first bin is (0,1], the second is (1,2], and so on) with the ml magnitude type and count how many are in each bin. 

In [45]:
# Separate out just the ml magType quakes
ml_quakes = quakes.query('magType == "ml"')

In [48]:
# what's the range we're looking at
min = ml_quakes.mag.min()
max = ml_quakes.mag.max()
print(f'Smallest mag is {min}, and largest is {max}')

Smallest mag is -1.26, and largest is 5.1


What's the deal with negative magnitude quakes? ...Did some research, apparently you can have negative magnitude quakes. Not a quirk of the dataset. Something about being less than a certain baseline, but it still registers to something humans can't feel.

In [98]:
bins = [-2, -1, 0, 1, 2, 3, 4, 5, 6, 7]
pd.cut(
    ml_quakes.mag, bins
).value_counts().sort_index()

(-2, -1]      13
(-1, 0]      491
(0, 1]      2207
(1, 2]      3105
(2, 3]       862
(3, 4]       122
(4, 5]         2
(5, 6]         1
(6, 7]         0
Name: mag, dtype: int64

Here's how the book solution did it:

In [81]:
# uses assign + lambda function to append the series that pd.cut makes to the
# main quakes dataframe, then value_counts that new mag_bin column.
quakes.query("magType == 'ml'").assign(
    mag_bin=lambda x: pd.cut(x.mag, np.arange(0, 10))
).mag_bin.value_counts()

(1, 2]    3105
(0, 1]    2207
(2, 3]     862
(3, 4]     122
(4, 5]       2
(5, 6]       1
(6, 7]       0
(7, 8]       0
(8, 9]       0
Name: mag_bin, dtype: int64

Hey look at that, the official solution misses some quakes that my approach captured. The output is also out of order. Bugs me a bit. It is more succinct tho. Let's combine the ideas from both:

In [99]:
# This is my final answer:
quakes.query("magType == 'ml'").assign(
    mag_bin=lambda x: pd.cut(x.mag, bins=np.arange(-2,8))
).mag_bin.value_counts().sort_index()

(-2, -1]      13
(-1, 0]      491
(0, 1]      2207
(1, 2]      3105
(2, 3]       862
(3, 4]       122
(4, 5]         2
(5, 6]         1
(6, 7]         0
Name: mag_bin, dtype: int64

---

## 3. Using the `faang.csv` file, group by the ticker and resample to monthly frequency. Make the following aggregations:
- Mean of the opening price
- Maximum of the high price
- Minimum of the low price
- Mean of the closing price
- Sum of the volume traded

In [116]:
# bring it in and check it out
#     setup the index as datetime for easy operations 
faang = pd.read_csv('exercises/faang.csv', index_col='date', parse_dates=True)
faang.head()

,ticker,high,low,open,close,volume
date,,,,,,
2018-01-02,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-03,FB,184.779999,181.330002,181.880005,184.669998,16886600.0
2018-01-04,FB,186.210007,184.100006,184.899994,184.330002,13880900.0
2018-01-05,FB,186.899994,184.929993,185.589996,186.850006,13574500.0
2018-01-08,FB,188.899994,186.330002,187.199997,188.279999,17994700.0


In [180]:
# groupby ticker
# resample the data by reducing granularity to monthly
# apply custom aggregations with agg

faang.groupby('ticker').resample('1M').agg({
    'open': 'mean',
    'high':'max',
    'low':'min',
    'close':'mean',
    'volume':'sum'
}).rename(columns={
    'open': 'openMean',
    'high': 'highMax',
    'low': 'lowMin',
    'close': 'closeMean',
    'volume': 'volumeSum',
}).head(24)


openMean      highMax       lowMin    closeMean  \
ticker date                                                             
AAPL   2018-01-31    43.505357    45.025002    41.174999    43.501309   
       2018-02-28    41.819079    45.154999    37.560001    41.909737   
       2018-03-31    43.761786    45.875000    41.235001    43.624048   
       2018-04-30    42.441310    44.735001    40.157501    42.458572   
       2018-05-31    46.239091    47.592499    41.317501    46.384205   
       2018-06-30    47.180119    48.549999    45.182499    47.155357   
       2018-07-31    47.549048    48.990002    45.855000    47.577857   
       2018-08-31    53.121739    57.217499    49.327499    53.336522   
       2018-09-30    55.582763    57.417500    53.825001    55.518421   
       2018-10-31    55.300000    58.367500    51.522499    55.211413   
       2018-11-30    47.954881    55.590000    42.564999    47.808929   
       2018-12-31    41.310789    46.235001    36.647499    41.066579   
AMZN   2018-01-31  1301.377151  1472.579956  1170.510010  1309.010946   
       2018-02-28  1447.113159  1528.699951  1265.930054  1442.363146   
       2018-03-31  1542.160464  1617.540039  1365.199951  1540.367629   
       2018-04-30  1475.841902  1638.099976  1352.880005  1468.220471   
       2018-05-31  1590.474543  1635.000000  1546.020020  1594.903637   
       2018-06-30  1699.088582  1763.099976  1635.089966  1698.823812   
       2018-07-31  1786.305716  1880.050049  1678.060059  1784.649042   
       2018-08-31  1891.957833  2025.569946  1776.020020  1897.851308   
       2018-09-30  1969.239476  2050.500000  1865.000000  1966.077900   
       2018-10-31  1799.630865  2033.189941  1476.359985  1782.058265   
       2018-11-30  1622.323806  1784.000000  1420.000000  1625.483823   
       2018-12-31  1572.922100  1778.339966  1307.000000  1559.443154   

                      volumeSum  
ticker date                      
AAPL   2018-01-31  2.638718e+09  
       2018-02-28  3.711577e+09  
       2018-03-31  2.854911e+09  
       2018-04-30  2.664617e+09  
       2018-05-31  2.483905e+09  
       2018-06-30  2.110498e+09  
       2018-07-31  1.574766e+09  
       2018-08-31  2.801276e+09  
       2018-09-30  2.715888e+09  
       2018-10-31  3.158994e+09  
       2018-11-30  3.845306e+09  
       2018-12-31  3.595690e+09  
AMZN   2018-01-31  9.637120e+07  
       2018-02-28  1.377840e+08  
       2018-03-31  1.304001e+08  
       2018-04-30  1.299196e+08  
       2018-05-31  7.161550e+07  
       2018-06-30  8.594130e+07  
       2018-07-31  9.752110e+07  
       2018-08-31  9.657580e+07  
       2018-09-30  9.444550e+07  
       2018-10-31  1.832208e+08  
       2018-11-30  1.392900e+08  
       2018-12-31  1.548127e+08

---

### 4. Build a crosstab with the earthquake data between the `tsunami` column and the `magType` column. Rather than showing freq count, show the max magnitude that was observed for each combo. Put the magnitude type along the columns 

Note: Crosstab is useful here because `.pivot_table()` only lets us do means, and we want to aggregate max for each combo.

In [195]:
# let's see it again
quakes.head()

,mag,magType,time,place,tsunami,parsed_place
0,1.35,ml,1539475168010,"9km NE of Aguanga, CA",0,California
1,1.29,ml,1539475129610,"9km NE of Aguanga, CA",0,California
2,3.42,ml,1539475062610,"8km NE of Aguanga, CA",0,California
3,0.44,ml,1539474978070,"9km NE of Aguanga, CA",0,California
4,2.16,md,1539474716050,"10km NW of Avenal, CA",0,California


In [209]:
pd.crosstab(
    index=quakes.tsunami,
    columns=quakes.magType,
    values=quakes.mag,
    aggfunc='max'
)

magType,mb,mb_lg,md,mh,ml,ms_20,mw,mwb,mwr,mww
tsunami,,,,,,,,,,
0,5.6,3.5,4.11,1.1,4.2,NaN,3.83,5.8,4.8,6.0
1,6.1,NaN,NaN,NaN,5.1,5.7,4.41,NaN,NaN,7.5


---

## 5. Calculate the rolling 60-day aggregations of the OHLC data by ticker for the FAANG data. Use the same aggregations as exercise 3.
```python
# reference
.agg({
    'open': 'mean',
    'high':'max',
    'low':'min',
    'close':'mean',
    'volume':'sum'})
```

In [228]:
faang.groupby('ticker').rolling('60D').agg({
    'open': 'mean',
    'high':'max',
    'low':'min',
    'close':'mean',
    'volume':'sum'
})

open        high         low       close       volume
ticker date                                                                   
AAPL   2018-01-02   42.540001   43.075001   42.314999   43.064999  102223600.0
       2018-01-03   42.836250   43.637501   42.314999   43.061249  220295200.0
       2018-01-04   42.935833   43.637501   42.314999   43.126666  310033600.0
       2018-01-05   43.041875   43.842499   42.314999   43.282499  404673600.0
       2018-01-08   43.151000   43.902500   42.314999   43.343500  486944800.0
...                       ...         ...         ...         ...          ...
NFLX   2018-12-24  283.509251  332.049988  233.679993  281.931750  525657600.0
       2018-12-26  281.844501  332.049988  231.229996  280.777750  520444300.0
       2018-12-27  281.070489  332.049988  231.229996  280.162927  532679500.0
       2018-12-28  279.916342  332.049988  231.229996  279.461464  521973500.0
       2018-12-31  278.430770  332.049988  231.229996  277.451539  476314900.0

[1255 rows x 5 columns]

---

## 6. Create a pivot table of the FAANG data that compares the stocks. Put the ticker in the rows, and show the averages of the OHLC and volume traded data.

In [229]:
faang.head()

,ticker,high,low,open,close,volume
date,,,,,,
2018-01-02,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-03,FB,184.779999,181.330002,181.880005,184.669998,16886600.0
2018-01-04,FB,186.210007,184.100006,184.899994,184.330002,13880900.0
2018-01-05,FB,186.899994,184.929993,185.589996,186.850006,13574500.0
2018-01-08,FB,188.899994,186.330002,187.199997,188.279999,17994700.0


In [230]:
faang.pivot_table(index='ticker')

,close,high,low,open,volume
ticker,,,,,
AAPL,47.263357,47.748526,46.795877,47.277859,1.360803e+08
AMZN,1641.726176,1662.839839,1619.840519,1644.072709,5.648994e+06
FB,171.510956,173.613347,169.303148,171.472948,2.765860e+07
GOOG,1113.225134,1125.777606,1101.001658,1113.554101,1.741965e+06
NFLX,319.290319,325.219322,313.187330,319.620558,1.146962e+07


---

## 7. Calculate the Z-scores for each numeric column of Amazon's (AMZN) data in Q4 2018 using apply().

Recall stuff:   
`.apply()` can run the same operation on all columns (or rows) of the df. Used in conjunction with a lambda function.  

Z-score, of course, is 
$ Z = \frac{X - \bar{X}}{S} $ 
  
So they come together like:  
`.apply(lambda x: x.sub(x.mean()).div(x.std())`
  
Now how exactly to do this...

In [242]:
faang.head()

,ticker,high,low,open,close,volume
date,,,,,,
2018-01-02,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-03,FB,184.779999,181.330002,181.880005,184.669998,16886600.0
2018-01-04,FB,186.210007,184.100006,184.899994,184.330002,13880900.0
2018-01-05,FB,186.899994,184.929993,185.589996,186.850006,13574500.0
2018-01-08,FB,188.899994,186.330002,187.199997,188.279999,17994700.0


In [243]:
# index is a datetime, so we can use .loc and cut out the requested year and quarter
# then query for just AMZN
# drop the ticker field because the Z score function is only for numeric columns
amzn = faang.loc['2018-q4'].query('ticker == "AMZN"').drop(['ticker'], axis=1)
amzn.head()

,high,low,open,close,volume
date,,,,,
2018-10-01,2033.189941,2003.599976,2021.989990,2004.359985,3460500.0
2018-10-02,2013.390015,1965.770020,1999.989990,1971.310059,5400700.0
2018-10-03,1989.699951,1949.810059,1981.699951,1952.760010,5253100.0
2018-10-04,1956.000000,1896.569946,1949.000000,1909.420044,7257000.0
2018-10-05,1929.079956,1862.829956,1917.989990,1889.650024,6822300.0


In [245]:
amzn.apply(lambda x: x.sub(x.mean()).div(x.std()))

,high,low,open,close,volume
date,,,,,
2018-10-01,2.368006,2.502113,2.337813,2.385848,-1.630411
2018-10-02,2.227302,2.247433,2.190795,2.155037,-0.861879
2018-10-03,2.058955,2.139987,2.068570,2.025489,-0.920345
2018-10-04,1.819474,1.781561,1.850048,1.722816,-0.126582
2018-10-05,1.628173,1.554416,1.642819,1.584748,-0.298771
...,...,...,...,...,...
2018-12-24,-2.159820,-2.187566,-2.179582,-2.226185,-0.141238
2018-12-26,-1.611714,-1.810493,-2.026617,-1.339674,1.123063
2018-12-27,-1.641276,-1.626703,-1.456521,-1.404343,0.849827


---

## 8. Add event descriptions
#### a) Create a df with the following three columns: `ticker`, `date`, `event`
the columns should have the following values:
- ticker: 'FB'
- date: ['2018-07-25', '2018-03-19', '2018-03-20']
- event: ['Disappointing user growth announced after close.', 'Cambridge Analytica story', 'FTC investigation']

#### b) Set the index to ['date', 'ticker']
#### c) Merge this data with the FAANG data using outer join.

In [277]:
fb_events = pd.DataFrame({
        'ticker': 'FB',
        'date': pd.to_datetime(['2018-07-25', '2018-03-19', '2018-03-20']),
        'event': ['Disappointing user growth announced after close.', 'Cambridge Analytica story', 'FTC investigation']
}).set_index(['ticker','date'])

fb_events


event
ticker date                                                        
FB     2018-07-25  Disappointing user growth announced after close.
       2018-03-19                         Cambridge Analytica story
       2018-03-20                                 FTC investigation

Doesn't seem to be a way to merge the elements in the `fb_events` hierarchical index to the, corresponding elements in `faang`.  
Rather, approach is to reindex `faang` to match that of `fb_events`.  
  
`.set_index()` takes existing columns. In the `faang` df, `date` is currently the index, by using 
reset_index() we push date to column status, then we can use `set_index()` to bring it back into
the index along with `ticker`.  
This works nice because `.join()` will detect and match index elements, just need to specify
the type of join. (Who did bid thee join with us?).  
  
Now we can do the outer join as requested:

In [296]:
events_faang = faang.reset_index().set_index(['date', 'ticker']).join(fb_events, how='outer')
events_faang

high          low         open        close  \
date       ticker                                                       
2018-01-02 AAPL      43.075001    42.314999    42.540001    43.064999   
           AMZN    1190.000000  1170.510010  1172.000000  1189.010010   
           FB       181.580002   177.550003   177.679993   181.419998   
           GOOG    1066.939941  1045.229980  1048.339966  1065.000000   
           NFLX     201.649994   195.419998   196.100006   201.070007   
...                        ...          ...          ...          ...   
2018-12-31 AAPL      39.840000    39.119999    39.632500    39.435001   
           AMZN    1520.760010  1487.000000  1510.800049  1501.969971   
           FB       134.639999   129.949997   134.449997   131.089996   
           GOOG    1052.699951  1023.590027  1050.959961  1035.609985   
           NFLX     270.100006   260.000000   260.160004   267.660004   

                        volume event  
date       ticker                     
2018-01-02 AAPL    102223600.0   NaN  
           AMZN      2694500.0   NaN  
           FB       18151900.0   NaN  
           GOOG      1237600.0   NaN  
           NFLX     10966900.0   NaN  
...                        ...   ...  
2018-12-31 AAPL    140014000.0   NaN  
           AMZN      6954500.0   NaN  
           FB       24625300.0   NaN  
           GOOG      1493300.0   NaN  
           NFLX     13508900.0   NaN  

[1255 rows x 6 columns]

So yeah, only 3 rows are gonna have the new data. We can see those:

In [299]:
events_faang[events_faang.notnull().all(1)]

,,high,low,open,close,volume,event
date,ticker,,,,,,
2018-03-19,FB,177.169998,170.059998,177.009995,172.559998,88140100.0,Cambridge Analytica story
2018-03-20,FB,170.199997,161.949997,167.470001,168.149994,129851800.0,FTC investigation
2018-07-25,FB,218.619995,214.270004,215.720001,217.500000,58954200.0,Disappointing user growth announced after close.


---

## 9. Use `transform()` on `faang` data to represent all the values in terms of the first date in the data. To do this, divide all values for each ticker by the values for the first date in the data for that ticker. 
Each resulting value is an **index**, and the data for the first date is the **base**  
Hint `transform()` can take a function name.

In [316]:
# break it out by ticker
faang_ticker = faang.reset_index().set_index(['ticker', 'date'])
faang_ticker.head()

high         low        open       close      volume
ticker date                                                                  
FB     2018-01-02  181.580002  177.550003  177.679993  181.419998  18151900.0
       2018-01-03  184.779999  181.330002  181.880005  184.669998  16886600.0
       2018-01-04  186.210007  184.100006  184.899994  184.330002  13880900.0
       2018-01-05  186.899994  184.929993  185.589996  186.850006  13574500.0
       2018-01-08  188.899994  186.330002  187.199997  188.279999  17994700.0

In [332]:
faang_ticker / faang_ticker.groupby(level='ticker').transform('first')

high       low      open     close    volume
ticker date                                                        
FB     2018-01-02  1.000000  1.000000  1.000000  1.000000  1.000000
       2018-01-03  1.017623  1.021290  1.023638  1.017914  0.930294
       2018-01-04  1.025498  1.036891  1.040635  1.016040  0.764708
       2018-01-05  1.029298  1.041566  1.044518  1.029931  0.747828
       2018-01-08  1.040313  1.049451  1.053579  1.037813  0.991340
...                     ...       ...       ...       ...       ...
GOOG   2018-12-24  0.940578  0.928131  0.928993  0.916638  1.284987
       2018-12-26  0.974750  0.940463  0.943406  0.976019  1.917663
       2018-12-27  0.978396  0.953857  0.970248  0.980169  1.704751
       2018-12-28  0.989334  0.988395  1.001221  0.973784  1.143180
       2018-12-31  0.986653  0.979296  1.002499  0.972404  1.206610

[1255 rows x 5 columns]

Do we need transform for this? 

In [327]:
faang_ticker / faang_ticker.groupby(level='ticker').first()

high       low      open     close    volume
ticker date                                                        
FB     2018-01-02  1.000000  1.000000  1.000000  1.000000  1.000000
       2018-01-03  1.017623  1.021290  1.023638  1.017914  0.930294
       2018-01-04  1.025498  1.036891  1.040635  1.016040  0.764708
       2018-01-05  1.029298  1.041566  1.044518  1.029931  0.747828
       2018-01-08  1.040313  1.049451  1.053579  1.037813  0.991340
...                     ...       ...       ...       ...       ...
GOOG   2018-12-24  0.940578  0.928131  0.928993  0.916638  1.284987
       2018-12-26  0.974750  0.940463  0.943406  0.976019  1.917663
       2018-12-27  0.978396  0.953857  0.970248  0.980169  1.704751
       2018-12-28  0.989334  0.988395  1.001221  0.973784  1.143180
       2018-12-31  0.986653  0.979296  1.002499  0.972404  1.206610

[1255 rows x 5 columns]

In [334]:
%timeit faang_ticker / faang_ticker.groupby(level='ticker').transform('first')

595 µs ± 5.67 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [335]:
%timeit faang_ticker / faang_ticker.groupby(level='ticker').first()

579 µs ± 2.62 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


So transform makes this a tad slower. Need to review uses for `transform()`

---

## 10. Data Processing for ECDC COVID-19 data. 
Snapshot of data thru 2020-09-18
  
### a) Prep the data:
- read in the data
- create a `date` column by parsing `dateRep` column to datetime and set as index
- convert text so we have the countries listed as `USA` and `UK`
- sort the index

In [1]:
import pandas as pd

In [3]:
# see the data as-is
covid = pd.read_csv('exercises/covid19_cases.csv')
covid.head()             

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
0,19/09/2020,19,9,2020,47,1,Afghanistan,AF,AFG,38041757.0,Asia,1.616645
1,18/09/2020,18,9,2020,0,0,Afghanistan,AF,AFG,38041757.0,Asia,1.535155
2,17/09/2020,17,9,2020,17,0,Afghanistan,AF,AFG,38041757.0,Asia,1.653446
3,16/09/2020,16,9,2020,40,10,Afghanistan,AF,AFG,38041757.0,Asia,1.708649
4,15/09/2020,15,9,2020,99,6,Afghanistan,AF,AFG,38041757.0,Asia,1.627159


In [49]:
# create the datetime index and do the requested string replacements
covid = pd.read_csv('exercises/covid19_cases.csv')\
    .assign(date=lambda x: pd.to_datetime(x.dateRep, format='%d/%m/%Y'))\
    .set_index('date')\
    .replace(['United_States_of_America', 'United_Kingdom'],['USA','UK'])\
    .sort_index()

covid.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
date,,,,,,,,,,,,
2019-12-31,31/12/2019,31,12,2019,0,0,Belgium,BE,BEL,11455519.0,Europe,NaN
2019-12-31,31/12/2019,31,12,2019,0,0,Mexico,MX,MEX,127575529.0,America,NaN
2019-12-31,31/12/2019,31,12,2019,0,0,Ecuador,EC,ECU,17373657.0,America,NaN
2019-12-31,31/12/2019,31,12,2019,0,0,Russia,RU,RUS,145872260.0,Europe,NaN
2019-12-31,31/12/2019,31,12,2019,0,0,Netherlands,NL,NLD,17282163.0,Europe,NaN


Alternate approach I had, does not leave the original dateRep column behind tho.

In [50]:
# handle the DatetimeIndex, and string replacement:
# covid = pd.read_csv('exercises/covid19_cases.csv', 
#                     parse_dates=['dateRep'], 
#                     index_col='dateRep',
#                     dayfirst=True,
#                     date_parser=lambda x: pd.to_datetime(x, format='%d/%m/%Y')
# ).replace(['United_States_of_America','United_Kingdom'],
#           ['USA', 'UK'])

### b) For the five countries with the most cases (cumulative), find the day with the largest number of cases.

In [72]:
covid.groupby('countriesAndTerritories')['cases']\
    .sum().reset_index().nlargest(5, 'cases')\
    ['countriesAndTerritories']

196       USA
92      India
27     Brazil
159    Russia
152      Peru
Name: countriesAndTerritories, dtype: object

In [75]:
# plug in the above to isolate the rows for just those countries
covid_highs = covid[covid['countriesAndTerritories'].isin(
        covid.groupby('countriesAndTerritories')['cases']\
        .sum().reset_index().nlargest(5, 'cases')['countriesAndTerritories'])]

covid_highs.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000,country_total_cases
date,,,,,,,,,,,,,
2019-12-31,31/12/2019,31,12,2019,0,0,Russia,RU,RUS,1.458723e+08,Europe,NaN,1091186
2019-12-31,31/12/2019,31,12,2019,0,0,USA,US,USA,3.290649e+08,America,NaN,6724667
2019-12-31,31/12/2019,31,12,2019,0,0,India,IN,IND,1.366418e+09,Asia,NaN,5308014
2019-12-31,31/12/2019,31,12,2019,0,0,Brazil,BR,BRA,2.110495e+08,America,NaN,4495183
2020-01-01,01/01/2020,1,1,2020,0,0,India,IN,IND,1.366418e+09,Asia,NaN,5308014


In [78]:
# groupby country and for each get the day with the largest caseload.
covid_highs.groupby('countriesAndTerritories')['cases'].nlargest(1)

countriesAndTerritories  date      
Brazil                   2020-07-30    69074
India                    2020-09-17    97894
Peru                     2020-08-17    10143
Russia                   2020-07-18    12640
USA                      2020-07-25    78427
Name: cases, dtype: int64

Great, my solution matches. Now let's see how the book did it...

In [81]:
# similar logic, but more conscise
covid_highs = covid\
    .groupby('countriesAndTerritories').cases.sum()\
    .nlargest(5).index


Index(['USA', 'India', 'Brazil', 'Russia', 'Peru'], dtype='object', name='countriesAndTerritories')

In [87]:
covid[covid.countriesAndTerritories.isin(covid_highs)]\
    .groupby('countriesAndTerritories').cases.idxmax()
# idxmax. Forgot about that. Returns the index with the highest values in rows.
# pandas has so much stuff.
# But my solution, you can see the numbers

countriesAndTerritories
Brazil   2020-07-30
India    2020-09-17
Peru     2020-08-17
Russia   2020-07-18
USA      2020-07-25
Name: cases, dtype: datetime64[ns]

---

### C) Find the 7-day average change in COVID-19 cases for the last week in the data for the five countries with the most cases.

In [90]:
# from the last section, we got the top 5:
list(covid_highs)

['USA', 'India', 'Brazil', 'Russia', 'Peru']

In [119]:
# Grouping by country again, but using the Grouper object we can hold onto all the days
# for each individual country, and for each we can now do a rolling avg:
covid_countries_by_day = covid.groupby(['countriesAndTerritories', pd.Grouper(freq='1D')])\
    ['cases'].sum().unstack(0)

covid_countries_by_day.head()

countriesAndTerritories,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua_and_Barbuda,Argentina,Armenia,Aruba,...,United_Republic_of_Tanzania,United_States_Virgin_Islands,Uruguay,Uzbekistan,Venezuela,Vietnam,Western_Sahara,Yemen,Zambia,Zimbabwe
date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2020-01-01,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2020-01-02,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2020-01-03,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2020-01-04,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [125]:
# this shows the seven day rolling average for the last week...
covid_countries_by_day.rolling(7).mean().last('1W')

countriesAndTerritories,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua_and_Barbuda,Argentina,Armenia,Aruba,...,United_Republic_of_Tanzania,United_States_Virgin_Islands,Uruguay,Uzbekistan,Venezuela,Vietnam,Western_Sahara,Yemen,Zambia,Zimbabwe
date,,,,,,,,,,,,,,,,,,,,,
2020-09-14,45.428571,156.857143,270.000000,18.428571,57.142857,0.0,0.0,10667.857143,154.142857,85.285714,...,0.0,5.571429,18.428571,549.285714,1035.857143,2.000000,0.0,3.714286,109.000000,78.428571
2020-09-15,45.857143,159.142857,263.285714,25.285714,60.428571,0.0,0.0,10963.571429,160.571429,82.571429,...,0.0,4.428571,17.000000,583.571429,1031.285714,2.000000,0.0,3.142857,126.285714,33.285714
2020-09-16,47.857143,159.857143,256.571429,25.285714,65.428571,0.0,0.0,11062.571429,166.571429,80.428571,...,0.0,5.857143,16.428571,602.714286,1013.142857,1.285714,0.0,3.142857,123.857143,26.857143
2020-09-17,46.857143,158.857143,250.000000,19.571429,76.571429,0.0,0.0,11043.428571,174.857143,85.428571,...,0.0,5.857143,16.428571,603.571429,952.142857,0.571429,0.0,3.000000,110.714286,24.142857
2020-09-18,42.857143,155.428571,243.714286,26.000000,93.857143,0.0,0.0,10959.857143,192.142857,80.428571,...,0.0,5.285714,16.714286,581.000000,923.000000,1.000000,0.0,2.857143,102.000000,25.714286
2020-09-19,44.714286,150.285714,237.285714,31.428571,81.714286,0.0,0.0,11073.571429,201.000000,80.285714,...,0.0,5.857143,16.714286,536.428571,930.142857,1.142857,0.0,2.428571,99.857143,24.000000


But we want to show the daily change of this rolling. `.diff` is the method for that.  
By default, this method shows the difference from the prior row:

In [127]:
covid_countries_by_day.diff().rolling(7).mean().last('1W')[covid_highs]

countriesAndTerritories,USA,India,Brazil,Russia,Peru
date,,,,,
2020-09-14,473.714286,181.285714,35.285714,36.285714,73.142857
2020-09-15,1513.000000,1142.857143,697.428571,46.285714,377.571429
2020-09-16,3478.714286,59.571429,3196.285714,61.428571,-65.000000
2020-09-17,-1047.000000,308.428571,143.428571,810.000000,-29.428571
2020-09-18,865.714286,-18.142857,-607.714286,-688.428571,-227.571429
2020-09-19,306.857143,-604.714286,-560.142857,57.285714,-41.285714


---

### d) Find the first date that each country other than China had cases.

In [133]:
covid.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000,country_total_cases
date,,,,,,,,,,,,,
2019-12-31,31/12/2019,31,12,2019,0,0,Belgium,BE,BEL,11455519.0,Europe,NaN,99555
2019-12-31,31/12/2019,31,12,2019,0,0,Mexico,MX,MEX,127575529.0,America,NaN,688954
2019-12-31,31/12/2019,31,12,2019,0,0,Ecuador,EC,ECU,17373657.0,America,NaN,124129
2019-12-31,31/12/2019,31,12,2019,0,0,Russia,RU,RUS,145872260.0,Europe,NaN,1091186
2019-12-31,31/12/2019,31,12,2019,0,0,Netherlands,NL,NLD,17282163.0,Europe,NaN,89959


In [156]:
covid.query('cases > 0 and countriesAndTerritories != "China"').reset_index()\
    .groupby('countriesAndTerritories')['date'].first()

countriesAndTerritories
Afghanistan      2020-02-25
Albania          2020-03-09
Algeria          2020-02-26
Andorra          2020-03-03
Angola           2020-03-22
                    ...    
Vietnam          2020-01-24
Western_Sahara   2020-04-26
Yemen            2020-04-10
Zambia           2020-03-19
Zimbabwe         2020-03-21
Name: date, Length: 209, dtype: datetime64[ns]

---

### e) Rank the countries by cumulative cases using percentiles

In [4]:
covid.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
0,19/09/2020,19,9,2020,47,1,Afghanistan,AF,AFG,38041757.0,Asia,1.616645
1,18/09/2020,18,9,2020,0,0,Afghanistan,AF,AFG,38041757.0,Asia,1.535155
2,17/09/2020,17,9,2020,17,0,Afghanistan,AF,AFG,38041757.0,Asia,1.653446
3,16/09/2020,16,9,2020,40,10,Afghanistan,AF,AFG,38041757.0,Asia,1.708649
4,15/09/2020,15,9,2020,99,6,Afghanistan,AF,AFG,38041757.0,Asia,1.627159


plan:
- put the summed cases by country into a table
- transform ranking by max as a percentage
- sort the rankings

In [9]:
# pivot table returns mean by default, here 
covid_case_totals = covid.pivot_table(
                        columns='countriesAndTerritories', 
                        values='cases', aggfunc='sum').T
covid_case_totals

,cases
countriesAndTerritories,
Afghanistan,38919
Albania,12073
Algeria,49413
Andorra,1564
Angola,3789
...,...
Vietnam,1068
Western_Sahara,766
Yemen,2024


In [17]:
# transforming and ranking by max setting pct=True will give the highest
# rank country a value of 1, and everything under that a relative proportion.
covid_case_totals = covid_case_totals.transform('rank', method='max', pct=True)
covid_case_totals

,cases
countriesAndTerritories,
Afghanistan,0.690476
Albania,0.561905
Algeria,0.733333
Andorra,0.271429
Angola,0.404762
...,...
Vietnam,0.228571
Western_Sahara,0.219048
Yemen,0.300000


In [18]:
covid_case_totals.sort_values('cases', ascending=False)

,cases
countriesAndTerritories,
United_States_of_America,1.000000
India,0.995238
Brazil,0.990476
Russia,0.985714
Peru,0.980952
...,...
Greenland,0.023810
Montserrat,0.019048
Falkland_Islands_(Malvinas),0.019048


---